<a href="https://colab.research.google.com/github/sudhakarmlal/EVA4/blob/master/S4/EVA4_Session_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,64, 3, padding=0, bias=False) 
        self.conv2 = nn.Conv2d(64, 16, 3, padding=0, bias=False)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1,bias=False)
        self.conv4 = nn.Conv2d(32, 10, 3, bias=False)
        self.batch1 = nn.BatchNorm2d(64)
        self.batch2 = nn.BatchNorm2d(16)
        self.batch3 = nn.BatchNorm2d(32)
        #self.batch3 = nn.BatchNorm2d(16)
        self.drop = nn.Dropout(0.05)
        self.gap = nn.AdaptiveAvgPool2d(1)

    def forward(self, x):
        x = self.drop(self.pool(self.batch1(F.relu(self.conv1(x)))))
        #print(x.shape)
        x = self.drop(self.pool(self.batch2(F.relu(self.conv2(x)))))
        #print(x.shape)
        x = self.drop(self.batch3(F.relu(self.conv3(x))))
        x = self.conv4(x)
        #print(x.shape)
        x = self.gap(x)
        x = x.view(-1, 10)
        #print(x.shape)
        return F.log_softmax(x)       

In [0]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 26, 26]             576
       BatchNorm2d-2           [-1, 64, 26, 26]             128
         MaxPool2d-3           [-1, 64, 13, 13]               0
           Dropout-4           [-1, 64, 13, 13]               0
            Conv2d-5           [-1, 16, 11, 11]           9,216
       BatchNorm2d-6           [-1, 16, 11, 11]              32
         MaxPool2d-7             [-1, 16, 5, 5]               0
           Dropout-8             [-1, 16, 5, 5]               0
            Conv2d-9             [-1, 32, 5, 5]           4,608
      BatchNorm2d-10             [-1, 32, 5, 5]              64
          Dropout-11             [-1, 32, 5, 5]               0
           Conv2d-12             [-1, 10, 3, 3]           2,880
AdaptiveAvgPool2d-13             [-1, 10, 1, 1]               0
Total params: 17,504
Trainable params: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:


torch.manual_seed(1)
batch_size = 16

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        #transforms.Normalize((0.2307,), (0.5081,))
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        #pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [0]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/3750 [00:00<?, ?it/s]


Test set: Average loss: 0.0375, Accuracy: 9881/10000 (98.81%)



  0%|          | 0/3750 [00:00<?, ?it/s]


Test set: Average loss: 0.0317, Accuracy: 9898/10000 (98.98%)



  0%|          | 0/3750 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0220, Accuracy: 9933/10000 (99.33%)



  0%|          | 0/3750 [00:00<?, ?it/s]


Test set: Average loss: 0.0219, Accuracy: 9939/10000 (99.39%)



100%|██████████| 3750/3750 [00:32<00:00, 114.74it/s]



Test set: Average loss: 0.0199, Accuracy: 9944/10000 (99.44%)

